In [1]:
from __future__ import unicode_literals
import pickle
import gensim
import pyLDAvis.gensim
import numexpr
import pyLDAvis
import pandas as pd
import spacy
pyLDAvis.enable_notebook()

In [2]:
text_blog = pd.read_csv('ivz_blog_corp.csv', error_bad_lines=False)

In [5]:
text_blog.to_pickle('./text.pkl')
text_blog = pd.read_pickle('./text.pkl')

In [7]:
# Text Parsing:stem and tokenize
texts_blog = gensim.parsing.preprocess_documents(text_blog['blogpost'].dropna())

In [8]:
# create bigram and trigram models
bigram_model_blog = gensim.models.Phrases(texts_blog, min_count=5, threshold=100)

trigram_model_blog = gensim.models.Phrases(bigram_model_blog[texts_blog], threshold = 100)

bigram_mod = gensim.models.phrases.Phraser(bigram_model_blog)
trigram_mod = gensim.models.phrases.Phraser(trigram_model_blog)

/Users/clairewillett/anaconda2/lib/python2.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [11]:
#functions: remove stopwords, create bigram, trigram, and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
# Remove Stop Words
import nltk
from gensim.utils import simple_preprocess
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/clairewillett/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
stop_words = stopwords.words('english')

texts_nostops = remove_stopwords(texts_blog)

In [35]:
#make bigrams 
bigrams_set = make_bigrams(texts_nostops)

In [36]:
#lemmatize, keeping only nouns, adjectives, verbs, and adverbs
nlp = spacy.load('en', disable=['parser', 'ner'])
lemma_set = lemmatization(bigrams_set, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [37]:
#make trigrams
trigrams_set = make_trigrams(lemma_set)

In [38]:
#create the dictionary and tfidf corpus (start w/ lemma bigrams)
dictionary_all = gensim.corpora.Dictionary(lemma_set)
corpus_bigrams = [dictionary_all.doc2bow(b) for b in lemma_set]

In [39]:
#bigrams model with 10 topics
lda_bi_10 = gensim.models.ldamodel.LdaModel(corpus = corpus_bigrams, id2word = dictionary_all, num_topics=10, passes=20)
lda_bi_10.save('invesco_bi10.model')

In [40]:
# Visualize the 10 topic bigram model
prepared_data_bi_10 = pyLDAvis.gensim.prepare(lda_bi_10, corpus_bigrams, dictionary_all)
prepared_data_bi_10

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      15.390760        1       1  0.070155 -0.237140
4      13.902688        1       2 -0.064938 -0.050601
1      11.952421        1       3 -0.084761 -0.013118
8      11.827356        1       4  0.006314  0.077364
0      11.689791        1       5 -0.054281  0.013725
2       9.406909        1       6  0.177971  0.093267
5       7.800716        1       7 -0.131611  0.026300
7       7.642817        1       8  0.052835 -0.015756
3       5.202315        1       9 -0.079545  0.059970
9       5.184228        1      10  0.107862  0.045990, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
725   Default   27.000000      altern   27.000000  30.0000  30.0000
166   Default  125.000000      market  125.000000  29.0000  29.0000
998   Default   22.000000  share_view   22.000000  28.0000  28.0000
336   Default   31.000000         fix   31.000000  27.0000  27.0000
87    Default   25.000000     currenc   25.000000  26.0000  26.0000
837   Default   29.000000     invesco   29.000000  25.0000  25.0000
973   Default   27.000000       world   27.000000  24.0000  24.0000
853   Default   35.000000       incom   35.000000  23.0000  23.0000
883   Default   23.000000      inflat   23.000000  22.0000  22.0000
77    Default   56.000000        rate   56.000000  21.0000  21.0000
1058  Default   35.000000     outlook   35.000000  20.0000  20.0000
952   Default   23.000000        save   23.000000  19.0000  19.0000
1135  Default   22.000000       retir   22.000000  18.0000  18.0000
7     Default   29.000000      dollar   29.000000  17.0000  17.0000
195   Default   24.000000         tax   24.000000  16.0000  16.0000
926   Default   46.000000      weekli   46.000000  15.0000  15.0000
1032  Default   17.000000      colleg   17.000000  14.0000  14.0000
587   Default   30.000000        plan   30.000000  13.0000  13.0000
1069  Default   21.000000    opportun   21.000000  12.0000  12.0000
513   Default   30.000000     compass   30.000000  11.0000  11.0000
673   Default   14.000000       yield   14.000000  10.0000  10.0000
270   Default   22.000000        rise   22.000000   9.0000   9.0000
803   Default   13.000000        fund   13.000000   8.0000   8.0000
321   Default   25.000000       trade   25.000000   7.0000   7.0000
334   Default   20.000000      polici   20.000000   6.0000   6.0000
34    Default   20.000000        risk   20.000000   5.0000   5.0000
627   Default   17.000000      impact   17.000000   4.0000   4.0000
242   Default   15.000000        high   15.000000   3.0000   3.0000
100   Default   24.000000         new   24.000000   2.0000   2.0000
1136  Default   19.000000      expect   19.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
225   Topic10    0.751794        meet    1.459389   2.2962  -5.7431
978   Topic10    2.118695        gift    4.191743   2.2772  -4.7070
99    Topic10    0.751794       hurdl    1.459389   2.2962  -5.7431
1149  Topic10    0.751794       exist    1.459389   2.2962  -5.7431
161   Topic10    2.118280      famili    4.381193   2.2328  -4.7072
414   Topic10    1.435246    concentr    2.900781   2.2559  -5.0965
723   Topic10    1.435162    approach    2.947747   2.2398  -5.0965
690   Topic10    1.435249    coverdel    2.956437   2.2369  -5.0965
83    Topic10    1.435248       index    2.964470   2.2342  -5.0965
776   Topic10    1.435150       trust    2.980552   2.2287  -5.0966
883   Topic10    6.896705      inflat   23.547002   1.7316  -3.5268
741   Topic10    2.093513        come    5.974381   1.9109  -4.7190
1179  Topic10    2.116978      consid    6.119355   1.8981  -4.7078
1015  Topic10    1.435177         esg    3.752633   1.9984  -5.0965
1032  Topic10    3.486130      colleg   17.505474   1.3458  -4.2090
977   Topic10    1.435253      measur    4.42

In [41]:
#create the trigram dictionary and tfidf corpus 
dictionary_tri = gensim.corpora.Dictionary(trigrams_set)
corpus_tri = [dictionary_tri.doc2bow(b) for b in trigrams_set]

In [44]:
lda_tri_9 = gensim.models.ldamodel.LdaModel(corpus = corpus_tri, id2word = dictionary_tri, num_topics=9, passes=20)
lda_tri_9.save('invesco_tri_9.model')

In [45]:
# Visualize the 9 topic trigram model
prepared_data_tri_9 = pyLDAvis.gensim.prepare(lda_tri_9, corpus_tri, dictionary_tri)
prepared_data_tri_9

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      15.542770        1       1 -0.077041  0.128203
4      15.424953        1       2  0.116082 -0.068968
2      12.439974        1       3 -0.109698 -0.063224
8      11.428688        1       4 -0.076751 -0.132938
0      10.672051        1       5  0.049220  0.000673
7      10.252879        1       6  0.210236  0.031736
1       9.303753        1       7 -0.044841  0.139548
3       7.724995        1       8 -0.024930 -0.068293
5       7.209936        1       9 -0.042277  0.033263, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
77    Default   55.000000        rate   55.000000  30.0000  30.0000
87    Default   23.000000     currenc   23.000000  29.0000  29.0000
725   Default   27.000000      altern   27.000000  28.0000  28.0000
837   Default   28.000000     invesco   28.000000  27.0000  27.0000
973   Default   26.000000       world   26.000000  26.0000  26.0000
7     Default   27.000000      dollar   27.000000  25.0000  25.0000
998   Default   21.000000  share_view   21.000000  24.0000  24.0000
336   Default   30.000000         fix   30.000000  23.0000  23.0000
166   Default  126.000000      market  126.000000  22.0000  22.0000
952   Default   23.000000        save   23.000000  21.0000  21.0000
926   Default   45.000000      weekli   45.000000  20.0000  20.0000
1186  Default   19.000000      invest   19.000000  19.0000  19.0000
587   Default   30.000000        plan   30.000000  18.0000  18.0000
853   Default   34.000000       incom   34.000000  17.0000  17.0000
1069  Default   21.000000    opportun   21.000000  16.0000  16.0000
883   Default   24.000000      inflat   24.000000  15.0000  15.0000
1058  Default   34.000000     outlook   34.000000  14.0000  14.0000
1032  Default   18.000000      colleg   18.000000  13.0000  13.0000
1135  Default   22.000000       retir   22.000000  12.0000  12.0000
270   Default   21.000000        rise   21.000000  11.0000  11.0000
513   Default   30.000000     compass   30.000000  10.0000  10.0000
242   Default   15.000000        high   15.000000   9.0000   9.0000
642   Default   22.000000      factor   22.000000   8.0000   8.0000
121   Default    8.000000        euro    8.000000   7.0000   7.0000
1136  Default   19.000000      expect   19.000000   6.0000   6.0000
673   Default   14.000000       yield   14.000000   5.0000   5.0000
730   Default   23.000000         kei   23.000000   4.0000   4.0000
324   Default    7.000000       japan    7.000000   3.0000   3.0000
195   Default   24.000000         tax   24.000000   2.0000   2.0000
708   Default   23.000000      reform   23.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
690    Topic9    1.522386    coverdel    3.056226   1.9328  -5.3669
875    Topic9    2.623034         oil    6.048166   1.7943  -4.8229
952    Topic9    7.291486        save   23.542255   1.4576  -3.8005
357    Topic9    2.243414     financi    6.227122   1.6088  -4.9792
1032   Topic9    4.406903      colleg   18.209651   1.2109  -4.3040
1005   Topic9    1.521629     surpris    3.828122   1.7071  -5.3674
500    Topic9    4.736106       stock   23.949195   1.0090  -4.2320
587    Topic9    5.122897        plan   30.997170   0.8295  -4.1535
214    Topic9    1.522381        hous    4.547031   1.5355  -5.3669
794    Topic9    1.522345       regul    4.667679   1.5093  -5.3670
161    Topic9    1.522386      famili    4.669278   1.5090  -5.3669
180    Topic9    1.522175      strike    4.718277   1.4984  -5.3671
1136   Topic9    2.964484      expect   19.101668   0.7666  -4.7005
334    Topic9    2.964729      polici   19.550362   0.7435  -4.7004
166    Topic9    6.694280      market  126.119842  -0.3063  -3.8860
926    Topic9    3.939221      weekli   45.972762   0.1726  -4.4162
1188   Topic9    1.961927      

In [46]:
print('Trigram Perplexity: ', lda_tri_9.log_perplexity(corpus_tri))  # a measure of how good the model is. lower the better.

(u'Trigram Perplexity: ', -6.816720598795075)


In [47]:
print('Bigram 10 Perplexity: ', lda_bi_10.log_perplexity(corpus_bigrams))  # a measure of how good the model is. lower the better.

(u'Bigram 10 Perplexity: ', -6.880233248373422)
